In [2]:
{
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Streaming Laptop Webcam to Web Interface\n",
                "\n",
                "This comprehensive guide will walk you through the process of capturing video from your laptop's webcam, publishing it as a ROS 2 topic, and streaming it to a web interface.\n",
                "\n",
                "## Overview\n",
                "\n",
                "The system consists of several components:\n",
                "1. **Camera Publisher Node** - Captures webcam video and publishes to ROS 2 topic\n",
                "2. **Web Bridge Node** - Subscribes to camera topic and streams to web interface\n",
                "3. **Web Application** - Displays the camera stream in a browser\n",
                "4. **Launch File** - Coordinates starting all nodes together\n",
                "\n",
                "## Prerequisites\n",
                "\n",
                "- ROS 2 (Humble or later)\n",
                "- Python 3.8+\n",
                "- OpenCV\n",
                "- WebSocket support\n",
                "- A laptop with a working webcam"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## Step 1: Create the Camera Publisher Node\n",
                "\n",
                "First, we need a ROS 2 node to access the webcam and publish the video feed. We'll create a new file for this in the `camera_stream_pkg`.\n",
                "\n",
                "### File: `src/camera_stream_pkg/camera_stream_pkg/camera_publisher.py`"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# camera_publisher.py\n",
                "import rclpy\n",
                "from rclpy.node import Node\n",
                "from sensor_msgs.msg import Image\n",
                "from cv_bridge import CvBridge\n",
                "import cv2\n",
                "import threading\n",
                "\n",
                "class CameraPublisher(Node):\n",
                "    def __init__(self):\n",
                "        super().__init__('camera_publisher')\n",
                "        \n",
                "        # Create publisher for camera images\n",
                "        self.image_publisher = self.create_publisher(Image, '/camera/image_raw', 10)\n",
                "        \n",
                "        # Initialize CV bridge for OpenCV <-> ROS image conversion\n",
                "        self.bridge = CvBridge()\n",
                "        \n",
                "        # Initialize camera capture\n",
                "        self.cap = cv2.VideoCapture(0)  # Use default camera\n",
                "        \n",
                "        if not self.cap.isOpened():\n",
                "            self.get_logger().error('Failed to open camera')\n",
                "            return\n",
                "        \n",
                "        # Set camera properties\n",
                "        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)\n",
                "        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)\n",
                "        self.cap.set(cv2.CAP_PROP_FPS, 30)\n",
                "        \n",
                "        # Create timer for publishing frames\n",
                "        self.timer = self.create_timer(0.033, self.publish_frame)  # ~30 FPS\n",
                "        \n",
                "        self.get_logger().info('Camera publisher node started')\n",
                "    \n",
                "    def publish_frame(self):\n",
                "        \"\"\"Capture and publish a camera frame\"\"\"\n",
                "        ret, frame = self.cap.read()\n",
                "        \n",
                "        if ret:\n",
                "            # Convert OpenCV image to ROS Image message\n",
                "            try:\n",
                "                ros_image = self.bridge.cv2_to_imgmsg(frame, encoding='bgr8')\n",
                "                ros_image.header.stamp = self.get_clock().now().to_msg()\n",
                "                ros_image.header.frame_id = 'camera_frame'\n",
                "                \n",
                "                # Publish the image\n",
                "                self.image_publisher.publish(ros_image)\n",
                "                \n",
                "            except Exception as e:\n",
                "                self.get_logger().error(f'Error converting/publishing image: {e}')\n",
                "        else:\n",
                "            self.get_logger().warn('Failed to capture frame from camera')\n",
                "    \n",
                "    def __del__(self):\n",
                "        \"\"\"Clean up camera resources\"\"\"\n",
                "        if hasattr(self, 'cap') and self.cap.isOpened():\n",
                "            self.cap.release()\n",
                "\n",
                "def main(args=None):\n",
                "    rclpy.init(args=args)\n",
                "    \n",
                "    camera_publisher = CameraPublisher()\n",
                "    \n",
                "    try:\n",
                "        rclpy.spin(camera_publisher)\n",
                "    except KeyboardInterrupt:\n",
                "        pass\n",
                "    finally:\n",
                "        camera_publisher.destroy_node()\n",
                "        rclpy.shutdown()\n",
                "\n",
                "if __name__ == '__main__':\n",
                "    main()"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## Step 2: Update Package Setup\n",
                "\n",
                "We need to update the `setup.py` file to include our new camera publisher node.\n",
                "\n",
                "### File: `src/camera_stream_pkg/setup.py`"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# setup.py modifications\n",
                "from setuptools import setup\n",
                "\n",
                "package_name = 'camera_stream_pkg'\n",
                "\n",
                "setup(\n",
                "    name=package_name,\n",
                "    version='0.0.0',\n",
                "    packages=[package_name],\n",
                "    data_files=[\n",
                "        ('share/ament_index/resource_index/packages',\n",
                "            ['resource/' + package_name]),\n",
                "        ('share/' + package_name, ['package.xml']),\n",
                "    ],\n",
                "    install_requires=['setuptools'],\n",
                "    zip_safe=True,\n",
                "    maintainer='your_name',\n",
                "    maintainer_email='your_email@example.com',\n",
                "    description='Camera streaming package for ROS 2',\n",
                "    license='Apache-2.0',\n",
                "    tests_require=['pytest'],\n",
                "    entry_points={\n",
                "        'console_scripts': [\n",
                "            'camera_publisher = camera_stream_pkg.camera_publisher:main',  # Updated entry point\n",
                "        ],\n",
                "    },\n",
                ")"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## Step 3: Update Web Bridge Node\n",
                "\n",
                "We need to modify the web bridge node to subscribe to the camera topic and stream images to the web interface.\n",
                "\n",
                "### File: `src/web_interface_pkg/web_interface_pkg/web_bridge_node.py`"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# web_bridge_node.py\n",
                "import rclpy\n",
                "from rclpy.node import Node\n",
                "from sensor_msgs.msg import Image\n",
                "from cv_bridge import CvBridge\n",
                "import cv2\n",
                "import asyncio\n",
                "import websockets\n",
                "import json\n",
                "import base64\n",
                "import threading\n",
                "import logging\n",
                "\n",
                "class WebBridgeNode(Node):\n",
                "    def __init__(self):\n",
                "        super().__init__('web_bridge_node')\n",
                "        \n",
                "        # Initialize CV bridge\n",
                "        self.bridge = CvBridge()\n",
                "        \n",
                "        # Subscribe to camera topic\n",
                "        self.image_subscription = self.create_subscription(\n",
                "            Image,\n",
                "            '/camera/image_raw',\n",
                "            self.image_callback,\n",
                "            10\n",
                "        )\n",
                "        \n",
                "        # Store latest frame\n",
                "        self.latest_frame = None\n",
                "        self.frame_lock = threading.Lock()\n",
                "        \n",
                "        # WebSocket server configuration\n",
                "        self.websocket_port = 8765\n",
                "        self.connected_clients = set()\n",
                "        \n",
                "        # Start WebSocket server in separate thread\n",
                "        self.websocket_thread = threading.Thread(target=self.start_websocket_server)\n",
                "        self.websocket_thread.daemon = True\n",
                "        self.websocket_thread.start()\n",
                "        \n",
                "        self.get_logger().info('Web bridge node started')\n",
                "        self.get_logger().info(f'WebSocket server starting on port {self.websocket_port}')\n",
                "    \n",
                "    def image_callback(self, msg):\n",
                "        \"\"\"Callback for received camera images\"\"\"\n",
                "        try:\n",
                "            # Convert ROS Image to OpenCV format\n",
                "            cv_image = self.bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')\n",
                "            \n",
                "            # Resize image for web streaming (optional)\n",
                "            cv_image = cv2.resize(cv_image, (640, 480))\n",
                "            \n",
                "            # Store the frame\n",
                "            with self.frame_lock:\n",
                "                self.latest_frame = cv_image.copy()\n",
                "                \n",
                "        except Exception as e:\n",
                "            self.get_logger().error(f'Error processing image: {e}')\n",
                "    \n",
                "    def start_websocket_server(self):\n",
                "        \"\"\"Start the WebSocket server\"\"\"\n",
                "        loop = asyncio.new_event_loop()\n",
                "        asyncio.set_event_loop(loop)\n",
                "        \n",
                "        start_server = websockets.serve(\n",
                "            self.websocket_handler,\n",
                "            'localhost',\n",
                "            self.websocket_port\n",
                "        )\n",
                "        \n",
                "        loop.run_until_complete(start_server)\n",
                "        loop.run_forever()\n",
                "    \n",
                "    async def websocket_handler(self, websocket, path):\n",
                "        \"\"\"Handle WebSocket connections\"\"\"\n",
                "        self.connected_clients.add(websocket)\n",
                "        self.get_logger().info(f'Client connected. Total clients: {len(self.connected_clients)}')\n",
                "        \n",
                "        try:\n",
                "            # Send frames to client\n",
                "            while True:\n",
                "                if self.latest_frame is not None:\n",
                "                    # Encode frame as JPEG\n",
                "                    with self.frame_lock:\n",
                "                        frame_copy = self.latest_frame.copy()\n",
                "                    \n",
                "                    _, buffer = cv2.imencode('.jpg', frame_copy, [cv2.IMWRITE_JPEG_QUALITY, 80])\n",
                "                    \n",
                "                    # Convert to base64 for web transmission\n",
                "                    frame_data = base64.b64encode(buffer).decode('utf-8')\n",
                "                    \n",
                "                    # Send frame data\n",
                "                    message = {\n",
                "                        'type': 'camera_frame',\n",
                "                        'data': frame_data\n",
                "                    }\n",
                "                    \n",
                "                    await websocket.send(json.dumps(message))\n",
                "                \n",
                "                # Control frame rate\n",
                "                await asyncio.sleep(0.033)  # ~30 FPS\n",
                "                \n",
                "        except websockets.exceptions.ConnectionClosed:\n",
                "            pass\n",
                "        except Exception as e:\n",
                "            self.get_logger().error(f'WebSocket error: {e}')\n",
                "        finally:\n",
                "            self.connected_clients.discard(websocket)\n",
                "            self.get_logger().info(f'Client disconnected. Total clients: {len(self.connected_clients)}')\n",
                "\n",
                "def main(args=None):\n",
                "    rclpy.init(args=args)\n",
                "    \n",
                "    web_bridge = WebBridgeNode()\n",
                "    \n",
                "    try:\n",
                "        rclpy.spin(web_bridge)\n",
                "    except KeyboardInterrupt:\n",
                "        pass\n",
                "    finally:\n",
                "        web_bridge.destroy_node()\n",
                "        rclpy.shutdown()\n",
                "\n",
                "if __name__ == '__main__':\n",
                "    main()"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## Step 4: Create Launch File\n",
                "\n",
                "We'll create a launch file to start both the camera publisher and web bridge node simultaneously.\n",
                "\n",
                "### File: `src/web_interface_pkg/launch/web_camera.launch.py`"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# web_camera.launch.py\n",
                "from launch import LaunchDescription\n",
                "from launch_ros.actions import Node\n",
                "from launch.actions import LogInfo\n",
                "\n",
                "def generate_launch_description():\n",
                "    return LaunchDescription([\n",
                "        LogInfo(msg=\"Starting camera streaming system...\"),\n",
                "        \n",
                "        # Launch camera publisher node\n",
                "        Node(\n",
                "            package='camera_stream_pkg',\n",
                "            executable='camera_publisher',\n",
                "            name='camera_publisher',\n",
                "            output='screen',\n",
                "            parameters=[\n",
                "                {'camera_index': 0},\n",
                "                {'frame_width': 640},\n",
                "                {'frame_height': 480},\n",
                "                {'fps': 30}\n",
                "            ]\n",
                "        ),\n",
                "        \n",
                "        # Launch web bridge node\n",
                "        Node(\n",
                "            package='web_interface_pkg',\n",
                "            executable='web_bridge_node',\n",
                "            name='web_bridge_node',\n",
                "            output='screen',\n",
                "            parameters=[\n",
                "                {'websocket_port': 8765}\n",
                "            ]\n",
                "        ),\n",
                "        \n",
                "        LogInfo(msg=\"Camera streaming system launched successfully!\")\n",
                "    ])"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## Step 5: Update Web Application\n",
                "\n",
                "We need to update the web application to connect to the WebSocket and display the camera stream.\n",
                "\n",
                "### File: `web_app/public/app.js`"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "// app.js\n",
                "class WebCameraInterface {\n",
                "    constructor() {\n",
                "        this.websocket = null;\n",
                "        this.isConnected = false;\n",
                "        this.reconnectInterval = 3000; // 3 seconds\n",
                "        this.maxReconnectAttempts = 10;\n",
                "        this.reconnectAttempts = 0;\n",
                "        \n",
                "        this.initializeElements();\n",
                "        this.connectWebSocket();\n",
                "    }\n",
                "    \n",
                "    initializeElements() {\n",
                "        // Get DOM elements\n",
                "        this.videoElement = document.getElementById('camera-feed');\n",
                "        this.statusElement = document.getElementById('connection-status');\n",
                "        this.reconnectButton = document.getElementById('reconnect-button');\n",
                "        \n",
                "        // Add event listeners\n",
                "        if (this.reconnectButton) {\n",
                "            this.reconnectButton.addEventListener('click', () => {\n",
                "                this.reconnectAttempts = 0;\n",
                "                this.connectWebSocket();\n",
                "            });\n",
                "        }\n",
                "        \n",
                "        console.log('Web interface initialized');\n",
                "    }\n",
                "    \n",
                "    connectWebSocket() {\n",
                "        try {\n",
                "            this.updateStatus('Connecting...', 'connecting');\n",
                "            \n",
                "            // Create WebSocket connection\n",
                "            this.websocket = new WebSocket('ws://localhost:8765');\n",
                "            \n",
                "            // Connection opened\n",
                "            this.websocket.onopen = (event) => {\n",
                "                console.log('WebSocket connection established');\n",
                "                this.isConnected = true;\n",
                "                this.reconnectAttempts = 0;\n",
                "                this.updateStatus('Connected', 'connected');\n",
                "            };\n",
                "            \n",
                "            // Message received\n",
                "            this.websocket.onmessage = (event) => {\n",
                "                try {\n",
                "                    const message = JSON.parse(event.data);\n",
                "                    this.handleMessage(message);\n",
                "                } catch (error) {\n",
                "                    console.error('Error parsing WebSocket message:', error);\n",
                "                }\n",
                "            };\n",
                "            \n",
                "            // Connection closed\n",
                "            this.websocket.onclose = (event) => {\n",
                "                console.log('WebSocket connection closed');\n",
                "                this.isConnected = false;\n",
                "                this.updateStatus('Disconnected', 'disconnected');\n",
                "                \n",
                "                // Attempt to reconnect\n",
                "                if (this.reconnectAttempts < this.maxReconnectAttempts) {\n",
                "                    this.reconnectAttempts++;\n",
                "                    console.log(`Attempting to reconnect... (${this.reconnectAttempts}/${this.maxReconnectAttempts})`);\n",
                "                    setTimeout(() => {\n",
                "                        this.connectWebSocket();\n",
                "                    }, this.reconnectInterval);\n",
                "                } else {\n",
                "                    this.updateStatus('Connection failed', 'error');\n",
                "                }\n",
                "            };\n",
                "            \n",
                "            // Error occurred\n",
                "            this.websocket.onerror = (error) => {\n",
                "                console.error('WebSocket error:', error);\n",
                "                this.updateStatus('Connection error', 'error');\n",
                "            };\n",
                "            \n",
                "        } catch (error) {\n",
                "            console.error('Error creating WebSocket connection:', error);\n",
                "            this.updateStatus('Connection error', 'error');\n",
                "        }\n",
                "    }\n",
                "    \n",
                "    handleMessage(message) {\n",
                "        switch (message.type) {\n",
                "            case 'camera_frame':\n",
                "                this.displayCameraFrame(message.data);\n",
                "                break;\n",
                "            \n",
                "            case 'status':\n",
                "                console.log('Status update:', message.data);\n",
                "                break;\n",
                "            \n",
                "            default:\n",
                "                console.log('Unknown message type:', message.type);\n",
                "        }\n",
                "    }\n",
                "    \n",
                "    displayCameraFrame(frameData) {\n",
                "        if (this.videoElement && frameData) {\n",
                "            // Create image URL from base64 data\n",
                "            const imageUrl = `data:image/jpeg;base64,${frameData}`;\n",
                "            \n",
                "            // Update image source\n",
                "            this.videoElement.src = imageUrl;\n",
                "        }\n",
                "    }\n",
                "    \n",
                "    updateStatus(message, statusClass) {\n",
                "        if (this.statusElement) {\n",
                "            this.statusElement.textContent = message;\n",
                "            this.statusElement.className = `status ${statusClass}`;\n",
                "        }\n",
                "        \n",
                "        // Show/hide reconnect button\n",
                "        if (this.reconnectButton) {\n",
                "            this.reconnectButton.style.display = \n",
                "                (statusClass === 'error' || statusClass === 'disconnected') ? 'inline-block' : 'none';\n",
                "        }\n",
                "    }\n",
                "    \n",
                "    disconnect() {\n",
                "        if (this.websocket) {\n",
                "            this.websocket.close();\n",
                "            this.websocket = null;\n",
                "        }\n",
                "        this.isConnected = false;\n",
                "    }\n",
                "}\n",
                "\n",
                "// Initialize the web interface when the page loads\n",
                "document.addEventListener('DOMContentLoaded', () => {\n",
                "    window.webCameraInterface = new WebCameraInterface();\n",
                "});\n",
                "\n",
                "// Clean up on page unload\n",
                "window.addEventListener('beforeunload', () => {\n",
                "    if (window.webCameraInterface) {\n",
                "        window.webCameraInterface.disconnect();\n",
                "    }\n",
                "});"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## Step 6: HTML Interface\n",
                "\n",
                "Here's the complete HTML interface for displaying the camera stream.\n",
                "\n",
                "### File: `web_app/public/index.html`"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "<!-- index.html -->\n",
                "<!DOCTYPE html>\n",
                "<html lang=\"en\">\n",
                "<head>\n",
                "    <meta charset=\"UTF-8\">\n",
                "    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">\n",
                "    <title>ROS 2 Camera Stream</title>\n",
                "    <style>\n",
                "        body {\n",
                "            font-family: Arial, sans-serif;\n",
                "            margin: 0;\n",
                "            padding: 20px;\n",
                "            background-color: #f0f0f0;\n",
                "        }\n",
                "        \n",
                "        .container {\n",
                "            max-width: 1200px;\n",
                "            margin: 0 auto;\n",
                "            background-color: white;\n",
                "            padding: 20px;\n",
                "            border-radius: 8px;\n",
                "            box-shadow: 0 2px 10px rgba(0,0,0,0.1);\n",
                "        }\n",
                "        \n",
                "        h1 {\n",
                "            color: #333;\n",
                "            text-align: center;\n",
                "            margin-bottom: 30px;\n",
                "        }\n",
                "        \n",
                "        .camera-section {\n",
                "            text-align: center;\n",
                "            margin-bottom: 30px;\n",
                "        }\n",
                "        \n",
                "        .camera-container {\n",
                "            display: inline-block;\n",
                "            border: 2px solid #ddd;\n",
                "            border-radius: 8px;\n",
                "            overflow: hidden;\n",
                "            background-color: #000;\n",
                "        }\n",
                "        \n",
                "        #camera-feed {\n",
                "            width: 640px;\n",
                "            height: 480px;\n",
                "            object-fit: cover;\n",
                "            display: block;\n",
                "        }\n",
                "        \n",
                "        .status-section {\n",
                "            text-align: center;\n",
                "            margin-bottom: 20px;\n",
                "        }\n",
                "        \n",
                "        .status {\n",
                "            padding: 10px 20px;\n",
                "            border-radius: 20px;\n",
                "            font-weight: bold;\n",
                "            display: inline-block;\n",
                "        }\n",
                "        \n",
                "        .status.connected {\n",
                "            background-color: #d4edda;\n",
                "            color: #155724;\n",
                "        }\n",
                "        \n",
                "        .status.connecting {\n",
                "            background-color: #fff3cd;\n",
                "            color: #856404;\n",
                "        }\n",
                "        \n",
                "        .status.disconnected {\n",
                "            background-color: #f8d7da;\n",
                "            color: #721c24;\n",
                "        }\n",
                "        \n",
                "        .status.error {\n",
                "            background-color: #f8d7da;\n",
                "            color: #721c24;\n",
                "        }\n",
                "        \n",
                "        #reconnect-button {\n",
                "            background-color: #007bff;\n",
                "            color: white;\n",
                "            border: none;\n",
                "            padding: 10px 20px;\n",
                "            border-radius: 5px;\n",
                "            cursor: pointer;\n",
                "            font-size: 16px;\n",
                "            margin-left: 10px;\n",
                "            display: none;\n",
                "        }\n",
                "        \n",
                "        #reconnect-button:hover {\n",
                "            background-color: #0056b3;\n",
                "        }\n",
                "        \n",
                "        .info-section {\n",
                "            background-color: #f8f9fa;\n",
                "            padding: 20px;\n",
                "            border-radius: 8px;\n",
                "            margin-top: 20px;\n",
                "        }\n",
                "        \n",
                "        .info-section h3 {\n",
                "            color: #495057;\n",
                "            margin-top: 0;\n",
                "        }\n",
                "        \n",
                "        .info-section p {\n",
                "            color: #6c757d;\n",
                "            line-height: 1.6;\n",
                "        }\n",
                "        \n",
                "        .placeholder {\n",
                "            width: 640px;\n",
                "            height: 480px;\n",
                "            background-color: #333;\n",
                "            display: flex;\n",
                "            align-items: center;\n",
                "            justify-content: center;\n",
                "            color: #fff;\n",
                "            font-size: 24px;\n",
                "        }\n",
                "    </style>\n",
                "</head>\n",
                "<body>\n",
                "    <div class=\"container\">\n",
                "        <h1>ROS 2 Camera Stream</h1>\n",
                "        <div class=\"status-section\">\n",
                "            <span id=\"connection-status\" class=\"status connecting\">Connecting...</span>\n",
                "            <button id=\"reconnect-button\">Reconnect</button>\n",
                "        </div>\n",
                "        <div class=\"camera-section\">\n",
                "            <div class=\"camera-container\">\n",
                "                <img id=\"camera-feed\" src=\"\" alt=\"Camera Feed\" class=\"placeholder\" />\n",
                "            </div>\n",
                "        </div>\n",
                "        <div class=\"info-section\">\n",
                "            <h3>Instructions</h3>\n",
                "            <p>\n",
                "                This web interface displays the live camera stream from your laptop's webcam via ROS 2 and WebSocket.<br>\n",
                "                Make sure the ROS 2 nodes and WebSocket server are running.<br>\n",
                "                If the stream does not appear, check your ROS 2 and WebSocket server logs, and try reconnecting.\n",
                "            </p>\n",
                "        </div>\n",
                "    </div>\n",
                "    <script src=\"app.js\"></script>\n",
                "</body>\n",
                "</html>\n"
            ]
        }
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "codemirror_mode": {
                "name": "ipython",
                "version": 3
            },
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.8.10"
        }
    },
    "nbformat": 4,
    "nbformat_minor": 4
}

NameError: name 'null' is not defined